In [1]:
import gc
import os
import time

import pandas as pd 
from datetime import datetime
import numpy as np

from numba import njit, prange
from itertools import combinations

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import *
from tensorflow.keras.activations import gelu, softmax
from tensorflow.keras.models import Sequential

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow_addons.activations import sparsemax
import random
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.layers import Input, Dropout, Dense, ReLU, BatchNormalization, Activation, Concatenate
from copy import deepcopy

import tensorflow.keras.layers as layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

# from tabtransformertf.utils.preprocessing import df_to_dataset, build_categorical_prep
# from tabtransformertf.models.fttransformer import FTTransformerEncoder, FTTransformer

import warnings
from warnings import simplefilter
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

import joblib
import functools

import gc
gc.collect();

EPS = 1e-8

2023-12-07 12:11:51.891322: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-07 12:11:51.910854: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 12:11:51.910873: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 12:11:51.910885: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 12:11:51.914962: I tensorflow/core/platform/cpu_feature_g

In [2]:
MODE = "train"  # train, inference, both
KAGGLE_DATASET_NAME = "fttransformer-version-yongmin-0"

In [3]:
if MODE == "train":
    print("You are in train mode")
    model_directory = "./models/" + time.strftime("%Y%m%d_%H:%M:%S", time.localtime(time.time() + 9 * 60 * 60))
    data_directory = "./data"
    train_mode = True
    infer_mode = False
elif MODE == "inference":
    print("You are in inference mode")
    model_directory = f'/kaggle/input/{KAGGLE_DATASET_NAME}'
    data_directory = "/kaggle/input/optiver-trading-at-the-close"
    train_mode = False
    infer_mode = True
elif MODE == "both":
    print("You are in both mode")
    model_directory = f'/kaggle/working/'
    data_directory = "/kaggle/input/optiver-trading-at-the-close"
    train_mode = True
    infer_mode = True
else:
    raise ValueError("Invalid mode")

You are in train mode


In [4]:
config = {
    "data_dir": data_directory,
    "model_dir": model_directory,

    "train_mode": train_mode,  # True : train, False : not train
    "infer_mode": infer_mode,  # True : inference, False : not inference
    "model_name": ["lgb"],  # model name
    "final_mode": False,  # True : using final model, False : not using final model
    "best_iterate_ratio": 1.2,  # best iteration ratio
    'target': 'target',

    'split_method': 'rolling',  # time_series, rolling, blocking, holdout
    'n_splits': 3,  # number of splits
    'correct': True,  # correct boundary
    'gap': 0.05,  # gap between train and test (0.05 = 5% of train size)

    'initial_fold_size_ratio': 0.8,  # initial fold size ratio
    'train_test_ratio': 0.9,  # train, test ratio

    'optuna_random_state': 42,
}
config["model_mode"] = "single" if len(config["model_name"]) == 1 else "stacking"  # 모델 수에 따라서 single / stacking 판단
config["mae_mode"] = True if config["model_mode"] == "single" and not config[
    "final_mode"] else False  # single 모델이면서 final_mode가 아닌경우 폴드가 여러개일때 모델 평가기준이 없어서 mae로 평가
config["inference_n_splits"] = len(config['model_name']) if config["final_mode"] or config["mae_mode"] else config[
    "n_splits"]  # final_mode가 아닌경우 n_splits만큼 inference

In [5]:
if MODE == "train":
    if not os.path.exists(config["model_dir"]):
        os.makedirs(config["model_dir"])
    if not os.path.exists(config["data_dir"]):
        os.makedirs(config["data_dir"])
    !kaggle competitions download optiver-trading-at-the-close -p {config["data_dir"]} --force
    !unzip -o {config["data_dir"]}/optiver-trading-at-the-close.zip -d {config["data_dir"]}
    !rm {config["data_dir"]}/optiver-trading-at-the-close.zip

100%|████████████████████████████████████████| 201M/201M [00:06<00:00, 33.2MB/s]
100%|████████████████████████████████████████| 201M/201M [00:06<00:00, 32.0MB/s]
Archive:  ./data/optiver-trading-at-the-close.zip
  inflating: ./data/example_test_files/revealed_targets.csv  
  inflating: ./data/example_test_files/sample_submission.csv  
  inflating: ./data/example_test_files/test.csv  
  inflating: ./data/optiver2023/__init__.py  
  inflating: ./data/optiver2023/competition.cpython-310-x86_64-linux-gnu.so  
  inflating: ./data/public_timeseries_testing_util.py  
  inflating: ./data/train.csv        


In [6]:
def print_log(message_format):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # self 확인: 첫 번째 인자가 클래스 인스턴스인지 확인합니다.
            if args and hasattr(args[0], 'infer'):
                self = args[0]

                # self.infer가 False이면 아무 것도 출력하지 않고 함수를 바로 반환합니다.
                if self.infer:
                    return func(*args, **kwargs)

            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()

            elapsed_time = end_time - start_time

            if result is not None:
                data_shape = getattr(result, 'shape', 'No shape attribute')
                shape_message = f", shape({data_shape})"
            else:
                shape_message = ""

            print(f"\n{'-' * 100}")
            print(message_format.format(func_name=func.__name__, elapsed_time=elapsed_time) + shape_message)
            print(f"{'-' * 100}\n")

            return result

        return wrapper

    return decorator


In [7]:
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """

    start_mem = df.memory_usage().sum() / 1024 ** 2

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)

    return df

In [8]:
class DataPreprocessor:

    def __init__(self, data, infer=False):
        self.data = memory_reduction.reduce_mem_usage(data)
        self.infer = infer

    @print_log("Executed {func.__name__} : shape({self.data.shape})")
    def handle_missing_data(self):
        # 결측치 처리 코드
        self.data = self.data.dropna(subset=["target"]) if self.infer == False else self.data
        self.data = self.data.fillna(0, columns = ['far_price', 'near_price'])
        self.data.reset_index(drop=True, inplace=True)
        return self.data

    @print_log("Executed {func.__name__} : shape({self.data.shape})")
    def handle_outliers(self):
        # 이상치 처리 코드
        return self.data

    @print_log("Executed {func.__name__} : shape({self.data.shape})")
    def normalize(self):
        # 정규화 코드
        return self.data

    @print_log("Executed {func.__name__} : shape({self.data.shape})")
    def custom_preprocessing(self):
        # 사용자 정의 전처리 코드
        return self.data

    @print_log("Executed {func.__name__} : shape({self.data.shape}) \n\n {self.data}")
    def transform(self):
        # 전처리 수행 코드 (위의 메소드 활용 가능)
        self.handle_missing_data()
        # self.handle_outliers()
        # self.normalize()
        # self.custom_preprocessing()
        return self.data

In [9]:
global_features = {}

In [10]:
def calculate_rsi(data, window_size=7):
    price_diff = data['wap'].diff()
    gain = price_diff.where(price_diff > 0, 0)
    loss = -price_diff.where(price_diff < 0, 0)

    avg_gain = gain.rolling(window=window_size).mean()
    avg_loss = loss.rolling(window=window_size).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

In [11]:
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    """
    Calculate the triplet imbalance for each row in the DataFrame.
    :param df_values: 
    :param comb_indices: 
    :return: 
    """
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            if mid_val == min_val:  # Prevent division by zero
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val + EPS)

    return imbalance_features


def calculate_triplet_imbalance_numba(price, df):
    """
    Calculate the triplet imbalance for each row in the DataFrame.
    :param price: 
    :param df: 
    :return: 
    """
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

In [12]:
class FeatureEngineer:

    def __init__(self, data, infer=False, feature_versions=None, dependencies=None,
                 base_directory="./data/fe_versions"):
        self.data = data
        self.infer = infer
        self.feature_versions = feature_versions or []
        self.dependencies = dependencies or {}  # 피처 버전 간 의존성을 정의하는 딕셔너리
        self.base_directory = base_directory
        if not os.path.exists(self.base_directory):
            os.makedirs(self.base_directory)

    @staticmethod
    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def generate_global_features(data):
        global_features["version_0"] = {
            "median_size": data.groupby("stock_id")["bid_size"].median() + data.groupby("stock_id")[
                "ask_size"].median(),
            "std_size": data.groupby("stock_id")["bid_size"].std() + data.groupby("stock_id")["ask_size"].std(),
            "ptp_size": data.groupby("stock_id")["bid_size"].max() - data.groupby("stock_id")["bid_size"].min(),
            "median_price": data.groupby("stock_id")["bid_price"].median() + data.groupby("stock_id")[
                "ask_price"].median(),
            "std_price": data.groupby("stock_id")["bid_price"].std() + data.groupby("stock_id")["ask_price"].std(),
            "ptp_price": data.groupby("stock_id")["bid_price"].max() - data.groupby("stock_id")["ask_price"].min(),
        }

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def feature_selection(self, data, exclude_columns):
        # 제외할 컬럼을 뺀 나머지로 구성된 새로운 DataFrame을 생성합니다.
        selected_columns = [c for c in data.columns if c not in exclude_columns]
        data = data[selected_columns]
        return data

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def feature_version_yongmin_0(self, *args, version_name="feature_version_yongmin_0"):
        
        df = pd.DataFrame(index=self.data.index)

        df['dow'] = self.data["date_id"] % 5
        df['seconds'] = self.data['seconds_in_bucket'] % 60
        df['minute'] = self.data['seconds_in_bucket'] // 60
    
        df["volume"] = self.data.eval("ask_size + bid_size")
        df['cum_wap'] = self.data.groupby(['stock_id'])['wap'].cumprod()
    
        for i in [1, 6]: # 1, 6도 생각해보기
            df[f'pct_change_{i}'] = self.data.groupby(['stock_id', 'seconds_in_bucket'])['wap'].pct_change(i).fillna(0)

            f = lambda x: 1 if x > 0 else (0 if x == 0 else -1)
            df[f'polarize_pct_{i}'] = df[f'pct_change_{i}'].apply(f)
    
        return df

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def feature_version_yongmin_1(self, *args, version_name="feature_version_yongmin_1"):
        
        df = pd.DataFrame(index=self.data.index)
        
        window_size = 6
        short_window = 1
        long_window = 6

        df['vol_st'] = self.data.groupby(['stock_id'])['wap'].pct_change().rolling(window=window_size).std()
        df['rolling_vol_di'] = self.data.groupby(['date_id'])['wap'].pct_change().rolling(window=window_size).std()
        df['std_st'] = self.data.groupby(['stock_id'])['wap'].rolling(window=window_size).std().values
        df['wap_pctch'] = self.data.groupby(['stock_id','date_id'])['wap'].pct_change().values*100
        df['short_ema'] = self.data.groupby(['stock_id'])['wap'].ewm(span=short_window, adjust=False).mean().values
        df['long_ema'] = self.data.groupby(['stock_id'])['wap'].ewm(span=long_window, adjust=False).mean().values
        wap_mean = self.data['wap'].mean()
        df['wap_vs_market'] = self.data['wap'] - self.data.groupby(['stock_id'])['wap'].transform('mean')
        df['macd'] = df['short_ema'] - df['long_ema']
        
        # Bollinger Bands calculation within each stock, date, and time
        df['bollinger_upper'] = self.data.groupby(['stock_id'])['wap'].rolling(window=long_window).mean().values + 2 * self.data.groupby(['stock_id'])['wap'].rolling(window=window_size).std().values
        df['bollinger_lower'] = self.data.groupby(['stock_id'])['wap'].rolling(window=long_window).mean().values - 2 * self.data.groupby(['stock_id'])['wap'].rolling(window=window_size).std().values
        # RSI calculation within each stock, date, and time
        df['rsi'] = self.data.groupby(['stock_id']).apply(calculate_rsi).values
        
        return df

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def feature_version_alvin_1(self, *args, version_name="feature_version_alvin_1"):
        # create empty dataframe
        df = pd.DataFrame(index=self.data.index)
        prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
        sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
        df["mid_price"] = self.data.eval("(ask_price + bid_price) / 2")
        df["liquidity_imbalance"] = self.data.eval(f"(bid_size-ask_size)/(bid_size+ask_size+{EPS})")
        df["matched_imbalance"] = self.data.eval(f"(imbalance_size-matched_size)/(matched_size+imbalance_size+{EPS})")
        df["size_imbalance"] = self.data.eval(f"bid_size / ask_size+{EPS}")

        for c in combinations(prices, 2):
            df[f"{c[0]}_{c[1]}_imb"] = self.data.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]} + {EPS})")

        for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
            triplet_feature = calculate_triplet_imbalance_numba(c, self.data)
            df[triplet_feature.columns] = triplet_feature.values

        return df

    # you can add more feature engineering version like above
    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def execute_feature_versions(self, save=False, load=False):
        results = {}

        for version in self.feature_versions:
            if load:
                df = self._load_from_parquet(version)
            else:
                method = getattr(self, version, None)
                if callable(method):
                    args = []
                    for dep in self.dependencies.get(version, []):
                        dep_result = results.get(dep)
                        if isinstance(dep_result, pd.DataFrame):
                            args.append(dep_result)
                        elif dep_result is None and hasattr(self, dep):
                            dep_method = getattr(self, dep)
                            dep_result = dep_method()
                            results[dep] = dep_result
                            args.append(dep_result)
                        else:
                            args.append(None)
                    df = method(*args)
                    if save:
                        self._save_to_parquet(df, version)
            results[version] = df

        # return that was in self.feature_versions
        return {k: v for k, v in results.items() if k in self.feature_versions}

    @print_log("Executed {func_name}, Elapsed time: {elapsed_time:.2f} seconds")
    def transform(self, save=False, load=False):
        feature_versions_results = self.execute_feature_versions(save=save, load=load)
        if not self.infer:
            self.data["date_id_copy"] = self.data["date_id"]
        concat_df = pd.concat([self.data] + list(feature_versions_results.values()), axis=1)

        exclude_columns = ["row_id", "time_id", "date_id"]
        final_data = self.feature_selection(concat_df, exclude_columns)
        final_data = concat_df
        return final_data


In [13]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

class QuantileTF:
    def __init__(self):
        self.scaler = QuantileTransformer(output_distribution='normal').set_output(transform="pandas")
        self.pipe = None

    def initialize_pipeline(self, df, usecols, passcols):
        columnTF = ColumnTransformer(
                                        transformers = [
                                            ("scaler" , self.scaler, usecols),
                                            ("pass", 'passthrough', passcols)
                                        ]
                                    )

        self.pipe = Pipeline([
                                ("scaler", columnTF.set_output(transform="pandas"))
                            ])

    def fit(self, df, usecols, passcols):
        self.initialize_pipeline(df, usecols, passcols)
        self.pipe.fit(df)
        
    def transform(self, df):
        cols = [col[8:] for col in self.pipe.get_feature_names_out() if "scaler" in col]
        return pd.DataFrame(self.pipe.transform(df), columns = cols)

    def fit_transform(self, df, usecols, passcols):
        self.initialize_pipeline(df, usecols, passcols)

        
        return self.pipe.fit_transform(df)

In [14]:
def df_to_dataset_(
    dataframe: pd.DataFrame,
    target: str = None,
    shuffle: bool = True,
    batch_size: int = 512,
):
    df = dataframe.copy()
    if target:
        labels = df.pop(target)
        # dataset = {}
        # for key, value in df.items():
        #     #dataset[key] = value[:, tf.newaxis] # old version
        #     dataset[key] = np.array(value)[:, tf.newaxis] # modified

        dataset = tf.data.Dataset.from_tensor_slices((df.to_numpy().astype(np.float32), labels))
    else:
        # dataset = {}
        # for key, value in df.items():
        #     #dataset[key] = value[:, tf.newaxis] # old version
        #     dataset[key] = np.array(value)[:, tf.newaxis] # modified

        dataset = tf.data.Dataset.from_tensor_slices(df.to_numpy().astype(np.float32))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    return dataset

# Model

In [15]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        # parametreleri
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        # batch-layer
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class FTTransformer(keras.Model):

    def __init__(self, 
            categories,
            num_continuous,
            dim,
            dim_out,
            depth,
            embedding_dim,
            heads,
            attn_dropout,
            ff_dropout,
            mlp_hidden,
            normalize_continuous = True):

        super(FTTransformer, self).__init__()

        # --> continuous inputs
        self.embedding_dim = embedding_dim
        self.normalize_continuous = normalize_continuous
        if normalize_continuous:
            self.continuous_normalization = layers.LayerNormalization()

        # --> categorical inputs

        # embedding
        self.embedding_layers = []
        for number_of_classes in categories:
            self.embedding_layers.append(layers.Embedding(input_dim = number_of_classes, output_dim = dim))
        self.embb = layers.Embedding(input_dim = 1000, output_dim = dim)
        self.flatten_output = layers.Flatten()

        # concatenation
        self.embedded_concatenation = layers.Concatenate(axis=1)
        self.cont_embedded_concatenation = layers.Concatenate(axis=1)

        # adding transformers
        self.transformers = []
        for _ in range(depth):
            self.transformers.append(TransformerBlock(dim, heads, dim))
        self.flatten_transformer_output = layers.Flatten()

        # --> MLP
        self.pre_concatenation = layers.Concatenate(axis=1)

        # mlp layers
        self.mlp_layers = []
        for size, activation in mlp_hidden:
            self.mlp_layers.append(layers.Dense(size, activation=activation))

        self.output_layer = layers.Dense(dim_out)

    def call(self, inputs):
        continuous_inputs  = inputs[0]
        categorical_inputs = inputs[1:]

        # --> continuous
        if self.normalize_continuous:
            continuous_inputs = self.continuous_normalization(continuous_inputs)

        continuous_inputs = self.embb(continuous_inputs)

        # --> categorical
        embedding_outputs = []
        for categorical_input, embedding_layer in zip(categorical_inputs, self.embedding_layers):
            embedding_outputs.append(embedding_layer(categorical_input))
        categorical_inputs = self.embedded_concatenation(embedding_outputs)
    
        trans_input = self.pre_concatenation([continuous_inputs, categorical_inputs])
        
        for transformer in self.transformers:
            trans_input = transformer(trans_input)
        mlp_input = self.flatten_transformer_output(trans_input)
        # --> MLP
        for mlp_layer in self.mlp_layers:
            mlp_input = mlp_layer(mlp_input)
        return self.output_layer(mlp_input)

In [16]:
def build_batch_data(x, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset_batch = dataset.batch(batch_size, drop_remainder = True)
    return dataset_batch

def get_X_from_groups(feature_set, groups):
    result = []
    for group in groups:
        result.append(feature_set[group])
    return result

def get_X_from_features(feature_set, cont_features, cat_features):
    groups = [cont_features]
    groups.extend(cat_features)
    return get_X_from_groups(feature_set, groups)

In [17]:
dependencies = {}

In [18]:
if config["train_mode"]:
    # Read Data
    df = (pd.read_csv(f"{config['data_dir']}/train.csv")
          .drop(['time_id', 'row_id'], axis=1))

    # Feature Engineering

    feature_engineer = FeatureEngineer(df, feature_versions=['feature_version_yongmin_0', 'feature_version_yongmin_1', 'feature_version_alvin_1'],
                                       dependencies=dependencies)
    
    feature_engineer.generate_global_features(df)
    
    df = feature_engineer.transform()
    
    # fillna
    df = (df.replace([np.inf, -np.inf], np.nan)
          .fillna(method='ffill')
          .fillna(0)
         )

    df = reduce_mem_usage(df)

    # Set Variable Type
    categorical = ['stock_id', 'date_id', 'imbalance_buy_sell_flag', 'polarize_pct_1', 'polarize_pct_6']
    continuous = list(set(df.columns) - set(categorical) - set(['target', 'date_id_copy']))
    features = categorical + continuous

    cats = [df[col].unique().shape[0] for col in categorical]

    scaler = QuantileTransformer(output_distribution = 'normal')
    
    rubbish = gc.collect()

    BATCH_SIZE = 64
    LR = 0.001
    EPOCH = 10
    MODEL_NAME = 'fttransformer_10epochV0.h5'

    ckp_path = os.path.join(config['model_dir'], MODEL_NAME)
    if not os.path.exists(config['model_dir']):
        os.mkdir(config['model_dir'])

    rlr = ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=3, verbose=0, min_delta=1e-5, mode='min')
    ckp = ModelCheckpoint(ckp_path, monitor='val_mae', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
    es = EarlyStopping(monitor='val_mae', min_delta=1e-4, patience=5, mode='min', restore_best_weights=True, verbose=0)

    model_callbacks = [rlr, ckp, es]
    rubbish = gc.collect()

    SPLIT_DAY = 390
    
    split = df['date_id'] > SPLIT_DAY
    df_train = df[~split]
    df_valid = df[split]


----------------------------------------------------------------------------------------------------
Executed generate_global_features, Elapsed time: 0.64 seconds
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
Executed feature_version_yongmin_0, Elapsed time: 2.43 seconds, shape((5237980, 9))
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
Executed feature_version_yongmin_1, Elapsed time: 11.11 seconds, shape((5237980, 11))
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
Executed feature_version_alvin_1, Elapsed time: 1.62 s

In [19]:
if config['train_mode']:
    # holdout
    X_train = df_train.drop(['target'], axis=1)
    y_train = df_train['target']

    X_test = df_valid.drop(['target'], axis=1)
    y_test = df_valid['target']
    
    if 'date_id_copy' in X_train.columns:
        X_train.drop(['date_id_copy'], axis=1)
        X_test.drop(['date_id_copy'], axis=1)
            
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    
    X_train[continuous] = scaler.fit_transform(X_train[continuous])
    X_test[continuous] = scaler.transform(X_test[continuous])
    print(f"{len(continuous)} Continuous Features Scaling Done")

    X_train = get_X_from_features(X_train, continuous, categorical)
    X_test = get_X_from_features(X_test, continuous, categorical)
    print("Dataset Transformed")

    model = FTTransformer(
                        categories = cats, # number of unique elements in each categorical feature
                        num_continuous = len(continuous),    # number of numerical features
                        dim = 16,                # embedding/transformer dimension
                        dim_out = 1,             # dimension of the model output
                        depth = 4,  
                        embedding_dim=32,       # number of transformer layers in the stack
                        heads = 6,               # number of attention heads
                        attn_dropout = 0.5,      # attention layer dropout in transformers
                        ff_dropout = 0.5,        # feed-forward layer dropout in transformers
                        mlp_hidden = [(512, 'relu'), (128, 'relu'), (64, 'relu'), (32, 'relu')]
                        #mlp_hidden = [(1024, 'relu'), (256, 'relu'), (128, 'relu'), (64, 'relu'), (32, 'relu')] # mlp layer dimensions and activations
                    )

    model.compile(loss = 'mae', 
                  metrics=['mae'], 
                  optimizer = tf.keras.optimizers.AdamW(LR))

    gc.collect();

    print("Model Training - HoldOut CV")
    model.fit(X_train, y_train,
              validation_data = (X_test, y_test),
              epochs=EPOCH, 
              batch_size=BATCH_SIZE,
              callbacks=model_callbacks)
        
    model.save(ckp_path)
    
    K.clear_session()
    del model
    rubbish = gc.collect()

(4247980, 62) (4247980,) (990000, 62) (990000,)
56 Continuous Features Scaling Done
Dataset Transformed


2023-12-07 12:13:12.102638: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 12:13:12.105343: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 12:13:12.105413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model Training - HoldOut CV
Epoch 1/10


2023-12-07 12:13:15.966798: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-12-07 12:13:16.622145: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f2218accaf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-07 12:13:16.622163: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-12-07 12:13:16.625882: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-07 12:13:16.670725: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


47323/66375 [====================>.........] - ETA: 11:23 - loss: 6.5026 - mae: 6.5026

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



66375/66375 [==============================] - 2620s 39ms/step - loss: 6.4977 - mae: 6.4977 - val_loss: 6.0216 - val_mae: 6.0216 - lr: 0.0010
Epoch 2/10
66375/66375 [==============================] - 1724s 26ms/step - loss: 6.4975 - mae: 6.4975 - val_loss: 6.0215 - val_mae: 6.0215 - lr: 0.0010
Epoch 3/10
66375/66375 [==============================] - 1345s 20ms/step - loss: 6.4975 - mae: 6.4975 - val_loss: 6.0220 - val_mae: 6.0220 - lr: 0.0010
Epoch 4/10
66375/66375 [==============================] - 1430s 22ms/step - loss: 6.4975 - mae: 6.4975 - val_loss: 6.0215 - val_mae: 6.0215 - lr: 0.0010
Epoch 5/10
13817/66375 [=====>........................] - ETA: 15:12 - loss: 6.4932 - mae: 6.4932

KeyboardInterrupt: 

In [ ]:
if MODE == "train":
    ! /usr/local/bin/kaggle datasets init -p {config['model_dir']}
    import json

    with open(f"{config['model_dir']}/dataset-metadata.json", "r") as file:
        data = json.load(file)

    data["title"] = data["title"].replace("INSERT_TITLE_HERE", f"{KAGGLE_DATASET_NAME}")
    data["id"] = data["id"].replace("INSERT_SLUG_HERE", f"{KAGGLE_DATASET_NAME}")

    with open(f"{config['model_dir']}/dataset-metadata.json", "w") as file:
        json.dump(data, file, indent=2)

    ! /usr/local/bin/kaggle datasets create -p {config['model_dir']}

    # !/usr/local/bin/kaggle datasets version -p {config['model_dir']} -m 'Updated data'

In [21]:
# df = (pd.read_csv(f"{config['data_dir']}/train.csv")
#       .drop(['time_id', 'row_id'], axis=1))

# # Feature Engineering

# feature_engineer = FeatureEngineer(df, feature_versions=['feature_version_yongmin_0', 'feature_version_yongmin_1'],
#                                    dependencies=dependencies)

# feature_engineer.generate_global_features(df)

# df = feature_engineer.transform()

# # fillna
# df = (df.replace([np.inf, -np.inf], np.nan)
#       .fillna(method='ffill')
#       .fillna(0)
#      )

# df = reduce_mem_usage(df)
# rubbish = gc.collect()

# BATCH_SIZE = 128
# LR = 0.001
# EPOCH = 30
# INPUT_SHAPE = df.shape[-1] - 2 # -2 for target and date_id_copy

# final_model = Sequential([
#                     Input(shape = (INPUT_SHAPE), batch_size=BATCH_SIZE),
#                     SAINT(3, 6, 8, 4, 4),
#                     Dense(1 , activation = 'linear')
# ])

# final_model.compile(tf.keras.optimizers.Adam(LR), 'mae', metrics=['mae'])

# model_path = "./models/20231130_11:05:26 /saint_30epoch.h5"
# final_model.load_weights(model_path)